In [1]:
import pickle
import re
import spacy
from spacy import displacy
import pandas as pd
import numpy as np
import pickle
import time
nlp = spacy.load("en_core_web_sm")

#=====================================================#
# Function for time
#=====================================================#
def convert(seconds): 
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
      
    return "%d:%02d:%02d" % (hour, minutes, seconds) 

"""

#=====================================================#
# Extract details from csv file
#=====================================================#
data = pd.read_csv('reviews.csv',index_col=[0])
data = data.fillna('-') # fill none with '-'
data.replace({'\n': ''}, inplace=True, regex=True) # remove break line

"""

#=====================================================#
# Extract details from pkl file
#=====================================================#
data = "Affordable Korean Food At Funan. Beef kalbi was disappointing, kimchi pancake is on the saltier side.... Fried chicken is normal..."
#data = data.fillna('-') # fill none with '-'
#data.replace({'\n': ''}, inplace=True, regex=True) # remove break line
#data['Review'] = data[['Title', 'Body']].apply(lambda x: '. '.join(x.astype(str)), axis=1) # join text from title and body
#data.head()


## Create column for food names per review

In [4]:
#=====================================================#
# Extract food names from text file
#=====================================================#
start = time.time()

with open('food_list_V2.txt',encoding="utf8") as f:
    food_list = [x.strip() for x in f.readlines()]

food_tag_list = []
for i in range(data.shape[0]):
    rev = data['Review'].iloc[i]
    #rev = rev.replace('and ', '').replace('& ', '')
    doc = nlp(rev)
    tag = []  
    food_tag = []
    
    # For each word in the review
    for food in food_list:
        if food in doc.text.lower():
            food_tag.append(food)
    food_tag_list.append(','.join(food_tag))

#=====================================================#
# Store food name into dataframe
#=====================================================#
data['Food'] = food_tag_list

print('Time taken:', convert(time.time()-start))

data.head()

Time taken: 1:14:27


,Title,Body,Reviewer,Reviewer_ID,Reviewer_Level,Reviewer_NumReviews,ReviewDateTime,Restaurant,Review,Food,RestaurantReview,FoodReview,RestaurantFeature(spacy),FoodFeature(spacy),RestaurantFeature(online),FoodFeature(online),RestaurantSentimentProba,FoodSentimentProba,OverallSentimentClass,Cuisine
0,Glad It’s Back!,We ordered two appetisers and four main dishes...,Sofina Ng,U0001,3.0,9.0,10m ago,/babasan-by-uncle-kiisu,Glad It’s Back!. We ordered two appetisers and...,"beer,fruit",glad it’s back!. we ordered two appetisers and...,kronenbourg blanc beer was very enjoyable too....,"ordered,main,unique,boasting,manifested,creati...","enjoyable,sweet,mild",unique,"enjoyable,fruity,mild,strong,sweet",0.981540,0.805150,1,NaN
1,Yuzu Ochazuke ($24),This rice in soup dish is the first one I’ve h...,Jan L,U0002,3.0,10.0,5d ago,/babasan-by-uncle-kiisu,Yuzu Ochazuke ($24). This rice in soup dish is...,rice,yuzu ochazuke ($24). boy did it impress! perfe...,this rice in soup dish is the first one i’ve h...,"impress,perfect,harmonious,fresh,generous,serv...","great,good","first,good,great,style","fresh,overpowering",0.963360,0.993274,1,NaN
2,Belly Roll Mee ($15),"The successor to Uncle Kiisu, Babasan’s menu d...",Dex Neo,U0003,8.0,958.0,Oct 24 at 5:42pm,/babasan-by-uncle-kiisu,Belly Roll Mee ($15). The successor to Uncle K...,"crispy pork,mee siam,pork,pork belly",belly roll mee ($15). the successor to uncle k...,i was seriously wondering how they would pull ...,"dreamt,good,sounds,heavy,provided,cut,amazingl...","seriously wondering,pull,crispy,stuffed,tender...","affordable,good,heavy","crispy,heavy,icy,rich,spicy,stuffed,tangy,tast...",0.986915,0.959834,1,NaN
3,Samurai Kueh Pie Tee,This was like a little bomb that exploded with...,Komal Salve,U0004,7.0,208.0,Oct 23 at 10:06am,/babasan-by-uncle-kiisu,Samurai Kueh Pie Tee. This was like a little b...,"fish,fish roe,kueh pie tee,pie,rice,roe,seafoo...",tee. this was like a little bomb that exploded...,samurai kueh pie i'm not a big fan of the seaf...,"little,exploded,eating,try,super worth","big,loved,flying,smooth","big,little","smooth,texture",0.959460,0.964552,1,NaN
4,Build Your Own Bowl (Small),Most salad/grain bowls serve up similar stuff ...,Nom Nom,U0005,3.0,15.0,Oct 25 at 10:13pm,/yobo,Build Your Own Bowl (Small). Most salad/grain ...,"brown rice,cream,pumpkin,rice,salad",build your own bowl (small). but i really enjo...,most salad/grain bowls serve up similar stuff ...,"build,small,enjoyed,served","salad,serve,similar,creamy,paired,brown,roaste...","good,small","creamy,roasted",0.915978,0.972140,1,NaN


## Create columns for restaurant and food reviews by splitting full reviews 

In [9]:
#=====================================================#
# Extract restaurant and food reviews
#=====================================================#
start = time.time()

with open('food_list_V2.txt',encoding="utf8") as f:
    food_list = [x.strip() for x in f.readlines()]

## Function set custom boundaries to break into phases or sentences
def set_custom_boundaries(doc):
    link_words = ['but','however','unlike', 'even though', 'despite', 'in spite of', 'unlike', 'whereas', 'while']
    for token in doc[:-1]:
        if token.text in link_words:
            doc[token.i].is_sent_start = True
    return doc

## Set spacy pipeline
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(set_custom_boundaries, before="parser")

## Get Restaurant and Food reviews
res_rev_list = []
food_rev_list = []

doc = nlp(data)
glist = []
flist = []
for sent in doc.sents: # for each phase/sentence
    glist.append(sent.text.lower()) # add all phase/sentence to generic list
    for s in food_list: # for each food name
        if s in sent.text.lower(): # check if food name appear in phase/sentence
            flist.append(sent.text.lower()) # add phase/sentence with food name to food list
            print('•', sent.text.lower())
            try:
                glist.remove(sent.text.lower()) # remove phase/sentence with food name from generic list
            except:
                pass
flist=list(dict.fromkeys(flist)) # remove duplicates
glist=list(dict.fromkeys(glist)) # remove duplicates
food_rev_list.append(' '.join(flist))
res_rev_list.append(' '.join(glist))


#=====================================================#
# Store segmented reviews into dataframe
#=====================================================#
#data['RestaurantReview'] = res_rev_list
#data['FoodReview'] = food_rev_list

#print('Time taken:', convert(time.time()-start))

#data.head()

• beef kalbi was disappointing, kimchi pancake is on the saltier side....
• beef kalbi was disappointing, kimchi pancake is on the saltier side....
• beef kalbi was disappointing, kimchi pancake is on the saltier side....
• fried chicken is normal...
• fried chicken is normal...


In [10]:
print('•', res_rev_list[0])

• affordable korean food at funan.
